# Assignment 5

This assignment has two parts. In the first part, you will apply the unsupervised learning techniques we discussed in class (dimensionality reduction and clustering) to understand patterns of health inequality in the United States. In the second part, you will apply algorithmic fairness techniques to study a widely used criminal risk prediction algorithm. 

In [1]:
# Load main libraries

import pandas as pd
import numpy as np

## Part 1: Unsupervised learning

1. The data we will be using comes from a paper entitled "The association between income and life expectancy in the United States, 2001-2014" (Chetty et al, JAMA 2016). You can download this paper [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4866586/pdf/nihms783419.pdf). Read the abstract and, in a few sentences, summarize the data it uses and the main conclusions it draws. (5 points)

2. As part of their paper, the authors released numerous aggregated datasets (yay!).  We will be using a dataset (health_ineq_online_table_12) which provides health and inequality metrics for each county. Because there are dozens of metrics, we would like to summarize them in some more compact way using unsupervised learning techniques. 

Please note that there are many missing values in the dataset. So we need to use the mean values to replace them. 

Let's start by preprocessing our dataset.

a. Remove all columns prior to cur_smoke_q1. Explain why it makes sense to exclude these columns. Then, remove all columns with at least 20% missing values. For the remaining columns, replace missing data in the column with the mean value in the column. Finally, normalize the dataset. Subtract the mean of each column and divide by its standard deviation so it will have mean zero and standard deviation 1 (10 points)

b. In a couple sentences, explain why it is important to put columns in this dataset on the same scale by dividing by the standard deviation before applying k-means or PCA. How much does the standard deviation of columns vary prior to standardizing them? (10 points)

In [2]:
df = pd.read_csv('health_ineq_online_table_12.csv', encoding = "ISO-8859-1")

FileNotFoundError: [Errno 2] No such file or directory: 'health_ineq_online_table_12.csv'

In [ ]:
df = df.drop(list(df)[0:14], axis=1)

In [ ]:
# Removing columns with at least 20% missing values

df = df.dropna(thresh=0.8*len(df), axis=1)

# Replace NaN with column mean

df.fillna(df.mean(), inplace=True)

# Normalizing the dataset

normalized_df = (df-df.mean())/df.std()

3. Apply PCA to the standardized dataset using sklearn. Print out how much variance is explained by each of the first 10 PCA dimensions, and the total fraction of variance explained. Write a couple sentences interpreting your results. (8 points)

In [ ]:
from sklearn.decomposition import PCA

model = PCA(n_components=10)
model.fit(normalized_df)

PCA(n_components=10)

In [ ]:
print("Proportion of variance explained by each component")
print(model.explained_variance_ratio_)

Proportion of variance explained by each component
[0.16840755 0.15682395 0.07454074 0.06367072 0.0478859  0.04279045
 0.03697806 0.03252388 0.02949627 0.02838289]


4. Plot the projection of each county using the first 2 PCA dimensions (ie, make a two-dimensional plot where each point is one county). Comment on any patterns you observe. (5 points)

5. Interpret the first two PCA dimensions by using the PCA component matrix (which you can get using fitted_model.components_, where fitted_model is your fitted PCA model) and looking at which of the dimensions in the original data the components correlate most strongly with (use the [codebook]('http://www.equality-of-opportunity.org/data/health/health_ineq_online_table_12_readme.pdf') to understand the meanings of different columns) (10 points)

6. Now let's try using k-means to cluster the data using sklearn.cluster.KMeans. We will be clustering data in the low-dimensional space (ie, using the first two PCA dimensions, not the original data). Using one of the methods discussed in the unsupervised learning lecture - for example, visual inspection, plotting the inertia, interpretability of the fitted clusters, etc - select a number of clusters (k). (We strongly suggest you use k<=5 to make it easier to interpret things.) This question is somewhat subjective, which is the point - unsupervised learning is often somewhat subjective - so write a couple sentences defending your method and choice of k. 

    Interpret the clusters by printing out the size of each cluster, and printing out cluster_centers_ as a dataframe with labeled rows and columns. To make the cluster centers dataframe easier to interpret, use df.style.background_gradient(cmap='RdBu') to color the entries of each cluster. We use the diverging colormap, RdBu, because it will use dramatic red-blue colors for both positive and negative entries, allowing us to see clusters that stand out in both directions. Write a couple sentences summarizing your findings.  (15 points)

## Part 2: Algorithmic fairness

This component of the assignment derives in part, with thanks and permission, from an [assignment](https://web.stanford.edu/class/cs182/assignments/AlgorithmicDecisionMaking-py.zip) in Stanford's CS182: Ethics, Public Policy, and Technological Change. Their assignment, in turn, is based on the journalistic organization ProPublica's [analysis](https://www.propublica.org/article/machine-bias-risk-assessments-in-criminal-sentencing) of a criminal risk prediction algorithm which we discussed in the algorithmic fairness lecture. Here, you will be assessing how a classifier designed to predict recidivism -- that is, whether someone will commit a crime in the future -- performs in terms of algorithmic fairness metrics. 

a. We have split the data for you into a train set (recidivism-training-data) and test set (recidivism-testing-data). You will be training the model on the train set and evaluating model predictions on the test set. (For simplicity, we do not use a validation set in this assignment because we will only be training a single model, so we do not need to select hyperparameters.) Read in the train set and test set, and read the data documentation in the "Algorithmic Fairness Data Documentation" file. (2 points)

In [5]:
train = pd.read_csv('recidivism-training-data.csv')
test = pd.read_csv('recidivism-testing-data.csv')

b. Use the train set to train an (unregularized) logistic regression model using sklearn.linear_model.LogisticRegression with penalty="none"¶, as in the regression assignment. Use the "recidivism_outcome" column as the variable you are trying to predict, and all the rest of the features as input features. 
Please note: there is no need for "patsy" or standardization, because the data is already in one-hot form (ie, coded as ones and zeroes) and we are not using regularization. (5 points)

In [6]:
targets = train['recidivism_outcome']
new_train = train.drop(['recidivism_outcome'], axis=1)

In [7]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(penalty='none')

logreg.fit(new_train, targets)

y_train_pred = logreg.predict(new_train)

c. Using the test set, report your model's AUC, false positive rate, false negative rate, and the fraction classified as positive, separately for white defendents and for Black defendents. In at least 5 sentences, describe what you observe, and any algorithmic fairness concerns it raises, making reference to algorithmic fairness concepts in class and using quantitative evidence as necessary. Do you believe this algorithm is fair enough to be deployed in practice? Why or why not? (15 points)

### White Defendants

In [8]:
test.head()

,Juvenile felony count = 0,Juvenile felony count = 1,Juvenile felony count = 2,Juvenile felony count >= 3,Juvenile misdemeanor count = 0,Juvenile misdemeanor count = 1,Juvenile misdemeanor count = 2,Juvenile misdemeanor count >= 3,Juvenile other offense count = 0,Juvenile other offense count = 1,...,Age > 45,Gender = Female,Gender = Male,Race = Other,Race = Asian,Race = Native American,Race = Caucasian,Race = Hispanic,Race = African American,recidivism_outcome
0,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,0
1,1,0,0,0,1,0,0,0,1,0,...,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,1,1
4,1,0,0,0,1,0,0,0,1,0,...,0,0,1,0,0,0,0,0,1,1


In [9]:
white_test = test[test['Race = Caucasian'] == 1]

In [10]:
# AUC

from sklearn.metrics import roc_auc_score
test_targets = white_test['recidivism_outcome']
run_white_test = white_test.drop(['recidivism_outcome'], axis=1)
y_test_pred = logreg.predict(run_white_test)

print('The AUC score of my White test set:')
roc_auc_score(test_targets, y_test_pred)

The AUC score of my White test set:


0.6631791077738516

In [11]:
# getting confusion matrix

from sklearn.metrics import confusion_matrix

# confusion matrix
matrix = confusion_matrix(test_targets,y_test_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(test_targets,y_test_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

Confusion matrix : 
 [[141 142]
 [ 77 371]]
Outcome values : 
 141 142 77 371


In [12]:
TP = 141
FP = 77
FN = 142
TN = 371

# apply confusion matrix
matrix_test_white = confusion_matrix(variables_test_white, y_test_pred_white, labels=[1,0])

# print matrix
print('Confusion matrix:')
print(matrix_test_white)

# get outcome values and reshape if values not same
tp, fn, fp, tn = confusion_matrix(variables_test_white, y_test_pred_white, labels=[1,0]).reshape(-1)

# print coutomes
print("Outcome values:")
print(tp, fn, fp, tn)

# calculate false positive rate and false negative rate
# source: https://www.split.io/glossary/false-positive-rate/

fp_rate = fp / (fp + tn)
fn_rate = fn / (fn + tp)

print('False positive rate: ', fp_rate)
print('False negative rate: ', fn_rate)

# calculate fraction classified as positive
# source: https://www.cuemath.com/numbers/positive-rational-numbers/

frac_class_as_positive = (tp + fp) / (tp + fp + fn + tn)
print('Fraction classified as positive: ', frac_class_as_positive)

In [13]:
# False positive rate: FP / (FP + TN)

print('The false positive rate is: ')
FP / (FP + TN)

The false positive rate is: 


0.171875

In [14]:
# False negative rate: FN / (FN + TP)

print('The false negative rate is: ')
FN / (FN + TP)

The false negative rate is: 


0.5017667844522968

In [15]:
# Fraction classified as positive

print('The fraction classified as positive is: ')
(TP + FP) / (TP + FP + FN + TN)

The fraction classified as positive is: 


0.2982216142270862

### Black Defendants

In [16]:
black_test = test[test['Race = African American'] == 1]

In [17]:
# AUC

from sklearn.metrics import roc_auc_score
test_targets = black_test['recidivism_outcome']
run_black_test = black_test.drop(['recidivism_outcome'], axis=1)
y_test_pred = logreg.predict(run_black_test)

print('The AUC score of my Black test set:')
roc_auc_score(test_targets, y_test_pred)

The AUC score of my Black test set:


0.6564179874558617

In [18]:
# getting confusion matrix

from sklearn.metrics import confusion_matrix

# confusion matrix
matrix = confusion_matrix(test_targets,y_test_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(test_targets,y_test_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

Confusion matrix : 
 [[415 154]
 [227 318]]
Outcome values : 
 415 154 227 318


In [19]:
TP = 415
FN = 154
FP = 227
TN = 318

In [20]:
# False positive rate: FP / (FP + TN)

print('The false positive rate is: ')
FP / (FP + TN)

The false positive rate is: 


0.41651376146788993

In [21]:
# False negative rate: FN / (FN + TP)

print('The false negative rate is: ')
FN / (FN + TP)

The false negative rate is: 


0.27065026362038663

In [22]:
# Fraction classified as positive

print('The fraction classified as positive is: ')
(TP + FP) / (TP + FP + FN + TN)

The fraction classified as positive is: 


0.5763016157989228

In terms of statistical parity (equal rates across groups), we observe that there is no statistical parity between Whites and Blacks. They have significantly different rates for all 3 measures. Additionally, in terms of predictive equality (equal false positive rates across groups), there is no equality either. Blacks have a much higher false positive rate than Whites, meaning that they are much more likely to be mis-labelled to commit a future crime. Whites also have a much higher false negative rate, meaning that many White defendants are being "missed" when they should be predicted for future crime. Both fairness principles are violated here.

We believe that this algorithm may still be fair enough to be deployed in practice for two main reasons. Firstly, additional fairness principles could still be explored, such as fairness through blindness (not including race) and using calibration (checking whether equal scores mean the same thing for both groups). Secondly, if this algorithm isn't used, it's still probably the case that the alternative solution (ie. humans or another model) are just as likely to commit bias as well. There is always a trade-off between fairness and maximizing accuracy, and no matter how much the model is finetuned, it will never be able to satisfy ALL of the fairness principles. So therefore, perhaps this model will not be able to improve by much, so we may as well use it since we already have the data.

d. Now train your own model, choosing the features you believe should be used (you are also welcome to use models which are not logistic regression models). Report the model's performance on white and Black defendents, using whatever metrics you believe are appropriate (you are also welcome to evaluate performance on other sensitive/protected groups). Write two paragraphs defending your model design choices, and explaining why you designed the model the way you did. (You're welcome to write two paragraphs explaining why you don't think models should be used in criminal risk prediction at all - this is a reasonable perspective! - but you still need to provide quantitative or non-quantitative evidence to back up your claims.) (15 points)

In [23]:
# feature selection

train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 42 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Juvenile felony count = 0                   5050 non-null   int64
 1   Juvenile felony count = 1                   5050 non-null   int64
 2   Juvenile felony count = 2                   5050 non-null   int64
 3   Juvenile felony count >= 3                  5050 non-null   int64
 4   Juvenile misdemeanor count = 0              5050 non-null   int64
 5   Juvenile misdemeanor count = 1              5050 non-null   int64
 6   Juvenile misdemeanor count = 2              5050 non-null   int64
 7   Juvenile misdemeanor count >= 3             5050 non-null   int64
 8   Juvenile other offense count = 0            5050 non-null   int64
 9   Juvenile other offense count = 1            5050 non-null   int64
 10  Juvenile other offense count = 2    

In [24]:
my_train = train[['Prior conviction count >= 3', 'Juvenile felony count >= 3', 'Charge degree = felony',
                 'Charge description = violent crime']]

In [25]:
targets = train['recidivism_outcome']

In [26]:
my_logreg = LogisticRegression(penalty='none')

my_logreg.fit(my_train, targets)

LogisticRegression(penalty='none')

### White Defendants with my model

In [27]:
# AUC

test_targets = white_test['recidivism_outcome']
white_test = white_test[['Prior conviction count >= 3', 'Juvenile felony count >= 3', 'Charge degree = felony',
                 'Charge description = violent crime']]
y_test_pred = my_logreg.predict(white_test)

print('The AUC score of my White test set:')
roc_auc_score(test_targets, y_test_pred)

The AUC score of my White test set:


0.6339758960121151

In [28]:
# confusion matrix
matrix = confusion_matrix(test_targets,y_test_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(test_targets,y_test_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

Confusion matrix : 
 [[139 144]
 [100 348]]
Outcome values : 
 139 144 100 348


In [29]:
TP = 139
FN = 144
FP = 100
TN = 348

In [30]:
# False positive rate: FP / (FP + TN)

print('The false positive rate is: ')
FP / (FP + TN)

The false positive rate is: 


0.22321428571428573

In [31]:
# False negative rate: FN / (FN + TP)

print('The false negative rate is: ')
FN / (FN + TP)

The false negative rate is: 


0.508833922261484

In [32]:
# Fraction classified as positive

print('The fraction classified as positive is: ')
(TP + FP) / (TP + FP + FN + TN)

The fraction classified as positive is: 


0.32694938440492477

### Black Defendants with my model

In [33]:
test_targets = black_test['recidivism_outcome']
black_test = black_test[['Prior conviction count >= 3', 'Juvenile felony count >= 3', 'Charge degree = felony',
                 'Charge description = violent crime']]
y_test_pred = my_logreg.predict(black_test)

print('The AUC score of my Black test set:')
roc_auc_score(test_targets, y_test_pred)

The AUC score of my Black test set:


0.6171183953822093

In [34]:
# confusion matrix
matrix = confusion_matrix(test_targets,y_test_pred, labels=[1,0])
print('Confusion matrix : \n',matrix)

# outcome values order in sklearn
tp, fn, fp, tn = confusion_matrix(test_targets,y_test_pred,labels=[1,0]).reshape(-1)
print('Outcome values : \n', tp, fn, fp, tn)

Confusion matrix : 
 [[340 229]
 [198 347]]
Outcome values : 
 340 229 198 347


In [35]:
TP = 340
FN = 229
FP = 198
TN = 347

In [36]:
# False positive rate: FP / (FP + TN)

print('The false positive rate is: ')
FP / (FP + TN)

The false positive rate is: 


0.363302752293578

In [37]:
# False negative rate: FN / (FN + TP)

print('The false negative rate is: ')
FN / (FN + TP)

The false negative rate is: 


0.4024604569420035

In [38]:
# Fraction classified as positive

print('The fraction classified as positive is: ')
(TP + FP) / (TP + FP + FN + TN)

The fraction classified as positive is: 


0.4829443447037702

### Synthesizing the results together, we see that:

**In the original model that includes all features**

WHITE
- AUC SCORE: 0.66
- False Positive Rate: 0.17
- False Negative Rate: 0.50
- Fraction classified as Positive: 0.30

BLACK
- AUC SCORE: 0.66
- False Positive Rate: 0.42
- False Negative Rate: 0.27
- Fraction classified as Positive: 0.58

**In my model that excludes race and only includes Prior Conviction Count >= 3, Juvenile Felony Count >= 3, Charge Degree = Felony, and Charge Description = Violent Crime**

WHITE
- AUC SCORE: 0.63
- False Positive Rate: 0.22
- False Negative Rate: 0.51
- Fraction classified as Positive: 0.33

BLACK
- AUC SCORE: 0.62
- False Positive Rate: 0.36
- False Negative Rate: 0.40
- Fraction classified as Positive: 0.48

In our model, we experimented with excluding race and gender to try and avoid introducing any sort of racial or gender bias in my model. This is "fairness through blindness", which may be a weak notion of fairness due to leaving out an important feature, but we wanted to experiment with this since the principles of both statistical parity and predictive equality were not maintained in the original model. We only included highly explanatory features that are directly related to crime: I believe they are highly associated with the possibility of re-committing crimes. This includes a high # of previous crimes (ie. Prior Conviction Count being >= 3), whether or not the defendant committed a lot of crimes when they were young (Juvenile Felony Count >= 3), the severity of the crime (Felony is more severe than Misdemeanour), and whether the crime was violent (Violent Crime). Because the defendant would have to be more courageous to have done a lot of historic and severe crimes, there's a larger chance they'd re-commit again. Also, we decided not to include the Counts that are <= 3 because we believe that defendants deserve a second or third chance to redeem themselves, but if a defendant commits 3 crimes, there is a greater chance they are less likely to reconsider their actions.

The AUC score slightly worsened after we tried our own model, but we can see from the false positive and false negative rates that the degree of algorithmic fairness has IMPROVED! Generally, both Blacks and Whites have their rates become closer to each other now. The false positive rate, which defines how often a defendant is mis-labeled as guilty to re-commit a crime, has become lower for Blacks and higher for Whites, making the two groups more equal to each other (although there is still a greater than 0.1 difference). There are also slightly more Whites being identified as positive than before. Unfortunately, the false negative rate has also increased for Blacks, which means that more Black defendants are not being "caught" correctly.  There are still many Whites, around the same amount as the prior model, that are also not being "caught"' correctly (which may potentially be due to their race or other features). Our model has increased the amount of defendants who are getting away with being identified for future crime. 

In summary, the two racial groups are now showing more similar false positive, false negative, and positive rates, but it's at the expense of a lower AUC score as well as a higher false negative rate for both groups, which means that more defendants are getting away with being identified for future crime. This is a trade-off in my model, and it points to the general idea that most models have trade-offs between algorithmic fairness and performance. 